# Invalidation Protocol

It would be helpful to know when cell output goes away so resources it was using can be cleaned up.

These resources might include:
* setInterval callbacks or setTimeout timer loops
* window or document-level event listeners
* WebGL stuff? dunno

This is along similar lines as a hypothetical "cell output is in view" or "cell output is visible (not collapsed)" protocol.

Here we load the shim in the same repo as this notebook: it uses IntersectionObservers and assumptions about the cell names in Jupyter Notebooks to call a function when the cell output div disappears.

In [1]:
%%html
<script src="https://unpkg.com/jupyter-invalidation/index.js"></script>

Now we do something interesting that takes global or finite resources. This could be a reasonable display of a kernel object, I'll write it inline for clarity.

This code logs mouseclicks to a div.

In [2]:
%%html
<div id="events" style="width: 600px; height 300px;"></div>
<script>
(function() {
    const onMouseDown = (e) => {
        events = document.getElementById('events');
        events.innerText = `${events.innerText} [${e.clientX}, ${e.clientY}]`;
        window.lastEvent = e;
    }
    window.addEventListener('mousedown', onMouseDown);
})();
</script>

If I rerun this cell twice, I get the effect twice.

I could stash some state in a global variable to ask the code to remove the old handler before adding the new one the next it runs, but that's both fragile (every kind of JavaScript repr needs its own global registry? Based on which cell it was rendered in?) and doesn't help in the cell deletion case, when I'd like to remove the effects without this code rerunning.

Here's a version using the invalidation hook:

In [3]:
%%html
<div id="events2" style="width: 600px; height 300px;"></div>
<script>
(function() {
    const onMouseDown = (e) => {
        events = document.getElementById('events2');
        events.innerText = `${events.innerText} [${e.clientX}, ${e.clientY}]`;
        window.lastEvent = e;
    }
    window.addEventListener('mousedown', onMouseDown);
    
    const cleanup = () => {
        console.log('cleaning up old handlers')
        window.removeEventListener('mousedown', onMouseDown);
    };
    
    onCellReexecuteOrDelete(document.getElementById('events2'), cleanup);
})();
</script>